# Pré-Processamento da Base de Dados - Instituições de Ensino Superior (IES):

### Importação das Bibliotecas:

In [4]:
import pandas as pd
import numpy as np

--------------------------------------------

### Obtenção dos Datasets:

In [8]:
df = pd.DataFrame()
for i in range(13):
    aux = pd.read_csv('../../data/trabalho2_dados'+str(i+1)+'.csv')
    df = pd.concat([df, aux])

df

,Codigo da IES,Natureza,Situacao,DIAS,%,CI N,CI V,IGC_N,IGC_M,IGC_V,EAD_17,variacao mat,mat_T 2017,% FIES,CURSOS,saldo 2017,Variacao 17-16
0,1298,1,Ativa,7313,0.739726,3,3,2.4867,2.4419,NaN,1,-0.089274,6284,0.299491,13,8.838117e+08,0.060665
1,282,0,Extinta,17065,0.000000,S/D,S/D,S/D,S/D,S/D,0,0.000000,0,0.000000,0,0.000000e+00,0.000000
2,264,0,Extinta,8879,0.000000,S/C,S/C,0,1.9828,1.81,0,0.000000,0,0.000000,0,0.000000e+00,0.000000
3,528,0,Ativa,2751,0.838710,5,S/C,3.8106,3.7309,3.7135,1,0.027620,12998,0.047777,47,-1.255788e+07,0.130986
4,3788,1,Ativa,4826,0.647059,3,5,2.6556,2.6556,2.6556,1,-0.484716,229,0.161572,6,7.997920e+07,0.580656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2172,1860,0,Ativa,664,0.705882,3,S/C,1.984,1.984,1.984,1,-0.597786,271,0.003690,5,-4.170386e+05,-0.189623
2173,19375,1,Ativa,616,0.714286,S/C,S/C,S/D,S/D,S/D,0,0.000000,0,0.000000,0,0.000000e+00,0.000000
2174,1183,0,Ativa,7510,0.000000,S/D,S/D,S/D,S/D,S/D,0,0.000000,0,0.000000,0,0.000000e+00,0.000000
2175,5430,0,Extinta,4162,0.000000,2,S/C,S/D,S/D,S/D,0,0.000000,0,0.000000,0,0.000000e+00,0.000000


----------------------------------------